In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import psycopg2.extras as extras
import pycountry

In [12]:
# načtení dat z CSV souboru
df = pd.read_csv("cizinci.CSV", sep=",", encoding="utf-8-sig")
# kontrola struktury dat
print(df)
# výběr relevantních sloupců
df = df[["pohlavi_txt", "vek_txt", "stobcan_txt", "rok"]]
print(df)
# přejmenování sloupců
df = df.rename(columns={
    "pohlavi_txt": "gender",
    "vek_txt": "age",
    "stobcan_txt": "country",
    "rok": "year"})
# kontrola struktury dat
print(df)
# odstranění prázdných hodnot
df = df.dropna()
# kontrola struktury dat
print(df)

             idhod  hodnota  stapro_kod  pohlavi_cis  pohlavi_kod   
0       1294742082   161711        6121          NaN          NaN  \
1       1294198145   342175        6121          NaN          NaN   
2       1294283124     3371        6121          NaN          NaN   
3       1294146027    15635        6121          NaN          NaN   
4       1294715182    25889        6121          NaN          NaN   
...            ...      ...         ...          ...          ...   
123305  1294301224        1        6121        102.0          1.0   
123306  1294191621        1        6121          NaN          NaN   
123307  1294199431        1        6121        102.0          2.0   
123308  1294191667        1        6121          NaN          NaN   
123309  1294191505        1        6121          NaN          NaN   

        stobcan_cis  stobcan_kod  vek_cis       vek_kod   rok  vuzemi_cis   
0            3228.0        804.0      NaN           NaN  2023         101  \
1               N

In [13]:
# přejmenování hodnot country na odpovídající kódy zemí
# Získání unikátních jmen zemí z datasetu
country_names = df['country'].unique()
print(f"Počet unikátních zemí v datasetu: {len(country_names)}")

# Vytvoření slovníku pro mapování českých názvů zemí na ISO kódy
country_mapping = {
    'Afghánistán': 'AFG',
    'Albánie': 'ALB',
    'Alžírsko': 'DZA',
    'Americká Samoa': 'ASM',
    'Americké Panenské ostrovy': 'VIR',
    'Andorra': 'AND',
    'Angola': 'AGO',
    'Antigua a Barbuda': 'ATG',
    'Argentina': 'ARG',
    'Arménie': 'ARM',
    'Aruba': 'ABW',
    'Austrálie': 'AUS',
    'Ázerbájdžán': 'AZE',
    'Bahamy': 'BHS',
    'Bahrajn': 'BHR',
    'Bangladéš': 'BGD',
    'Barbados': 'BRB',
    'Belgie': 'BEL',
    'Belize': 'BLZ',
    'Bělorusko': 'BLR',
    'Benin': 'BEN',
    'Bermudy': 'BMU',
    'Bhútán': 'BTN',
    'Bolívie': 'BOL',
    'Bosna a Hercegovina': 'BIH',
    'Botswana': 'BWA',
    'Brazílie': 'BRA',
    'Britské indickooceánské území': 'IOT',
    'Brunej': 'BRN',
    'Bulharsko': 'BGR',
    'Burkina Faso': 'BFA',
    'Burundi': 'BDI',
    'Cookovy ostrovy': 'COK',
    'Čad': 'TCD',
    'Černá Hora': 'MNE',
    'Česká republika': 'CZE',
    'Česko': 'CZE',
    'Čína': 'CHN',
    'Dánsko': 'DNK',
    'Dominika': 'DMA',
    'Dominikánská republika': 'DOM',
    'Džibutsko': 'DJI',
    'Egypt': 'EGY',
    'Ekvádor': 'ECU',
    'Eritrea': 'ERI',
    'Estonsko': 'EST',
    'Etiopie': 'ETH',
    'Faerské ostrovy': 'FRO',
    'Falklandy': 'FLK',
    'Fidži': 'FJI',
    'Filipíny': 'PHL',
    'Finsko': 'FIN',
    'Francie': 'FRA',
    'Francouzská Guyana': 'GUF',
    'Francouzská Polynésie': 'PYF',
    'Francouzská jižní území': 'ATF',
    'Gabon': 'GAB',
    'Gambie': 'GMB',
    'Ghana': 'GHA',
    'Gibraltar': 'GIB',
    'Grenada': 'GRD',
    'Gruzie': 'GEO',
    'Grónsko': 'GRL',
    'Guadeloupe': 'GLP',
    'Guam': 'GUM',
    'Guatemala': 'GTM',
    'Guinea': 'GIN',
    'Guinea-Bissau': 'GNB',
    'Guyana': 'GUY',
    'Haiti': 'HTI',
    'Honduras': 'HND',
    'Hongkong': 'HKG',
    'Chile': 'CHL',
    'Chorvatsko': 'HRV',
    'Indie': 'IND',
    'Indonésie': 'IDN',
    'Irák': 'IRQ',
    'Írán': 'IRN',
    'Irsko': 'IRL',
    'Island': 'ISL',
    'Itálie': 'ITA',
    'Izrael': 'ISR',
    'Jamajka': 'JAM',
    'Japonsko': 'JPN',
    'Jemen': 'YEM',
    'Jihoafrická republika': 'ZAF',
    'Jižní Korea': 'KOR',
    'Jižní Súdán': 'SSD',
    'Jordánsko': 'JOR',
    'Kajmanské ostrovy': 'CYM',
    'Kambodža': 'KHM',
    'Kamerun': 'CMR',
    'Kanada': 'CAN',
    'Kapverdy': 'CPV',
    'Katar': 'QAT',
    'Kazachstán': 'KAZ',
    'Keňa': 'KEN',
    'Kiribati': 'KIR',
    'Kolumbie': 'COL',
    'Komory': 'COM',
    'Kongo': 'COG',
    'Kongo (Brazzaville)': 'COG',
    'Kongo (Kinshasa)': 'COD',
    'Kostarika': 'CRI',
    'Kosovo': 'XKX',
    'Kuba': 'CUB',
    'Kuvajt': 'KWT',
    'Kypr': 'CYP',
    'Kyrgyzstán': 'KGZ',
    'Laos': 'LAO',
    'Lesotho': 'LSO',
    'Libanon': 'LBN',
    'Libérie': 'LBR',
    'Libye': 'LBY',
    'Lichtenštejnsko': 'LIE',
    'Lichtejnštejnsko': 'LIE',
    'Litva': 'LTU',
    'Lotyšsko': 'LVA',
    'Lucembursko': 'LUX',
    'Macao': 'MAC',
    'Madagaskar': 'MDG',
    'Maďarsko': 'HUN',
    'Makedonie': 'MKD',
    'Malajsie': 'MYS',
    'Malawi': 'MWI',
    'Maledivy': 'MDV',
    'Maldivská republika': 'MDV',
    'Mali': 'MLI',
    'Malta': 'MLT',
    'Man (ostrov)': 'IMN',
    'Maroko': 'MAR',
    'Marshallovy ostrovy': 'MHL',
    'Martinik': 'MTQ',
    'Mauritánie': 'MRT',
    'Mauritius': 'MUS',
    'Mayotte': 'MYT',
    'Mexiko': 'MEX',
    'Mikronésie': 'FSM',
    'Federativní státy Mikronésie': 'FSM',
    'Moldavsko': 'MDA',
    'Moldávie': 'MDA',
    'Monako': 'MCO',
    'Mongolsko': 'MNG',
    'Montserrat': 'MSR',
    'Mosambik': 'MOZ',
    'Myanmar': 'MMR',
    'Barma': 'MMR',
    'Namibie': 'NAM',
    'Nauru': 'NRU',
    'Německo': 'DEU',
    'Nepál': 'NPL',
    'Niger': 'NER',
    'Nigérie': 'NGA',
    'Nikaragua': 'NIC',
    'Niue': 'NIU',
    'Nizozemí': 'NLD',
    'Nizozemsko': 'NLD',
    'Norfolk': 'NFK',
    'Norsko': 'NOR',
    'Nová Kaledonie': 'NCL',
    'Nový Zéland': 'NZL',
    'Omán': 'OMN',
    'Pákistán': 'PAK',
    'Palau': 'PLW',
    'Palestina': 'PSE',
    'Stát Palestina': 'PSE',
    'Panama': 'PAN',
    'Papua-Nová Guinea': 'PNG',
    'Paraguay': 'PRY',
    'Peru': 'PER',
    'Pitcairn': 'PCN',
    'Pobřeží slonoviny': 'CIV',
    'Polsko': 'POL',
    'Portoriko': 'PRI',
    'Portugalsko': 'PRT',
    'Rakousko': 'AUT',
    'Réunion': 'REU',
    'Rovníková Guinea': 'GNQ',
    'Rumunsko': 'ROU',
    'Rusko': 'RUS',
    'Rwanda': 'RWA',
    'Řecko': 'GRC',
    'Saint Pierre a Miquelon': 'SPM',
    'Saint Kitts a Nevis': 'KNA',
    'Svatý Kryštof a Nevis': 'KNA',
    'Salvador': 'SLV',
    'Samoa': 'WSM',
    'San Marino': 'SMR',
    'Saúdská Arábie': 'SAU',
    'Senegal': 'SEN',
    'Severní Korea': 'PRK',
    'Severní Makedonie': 'MKD',
    'Seychely': 'SYC',
    'Sierra Leone': 'SLE',
    'Singapur': 'SGP',
    'Slovensko': 'SVK',
    'Slovinsko': 'SVN',
    'Somálsko': 'SOM',
    'Spojené arabské emiráty': 'ARE',
    'Spojené státy americké': 'USA',
    'Srbsko': 'SRB',
    'Srí Lanka': 'LKA',
    'Středoafrická republika': 'CAF',
    'Súdán': 'SDN',
    'Surinam': 'SUR',
    'Svatá Helena': 'SHN',
    'Svatá Lucie': 'LCA',
    'Svatý Tomáš a Princův ostrov': 'STP',
    'Sv. Tomáš a Princův ostrov': 'STP',
    'Svatý Vincent': 'VCT',
    'Svatý Vincenc a Grenadiny': 'VCT',
    'Svazijsko': 'SWZ',
    'Eswatini': 'SWZ',
    'Sýrie': 'SYR',
    'Šalamounovy ostrovy': 'SLB',
    'Španělsko': 'ESP',
    'Švédsko': 'SWE',
    'Švýcarsko': 'CHE',
    'Tádžikistán': 'TJK',
    'Tanzanie': 'TZA',
    'Thajsko': 'THA',
    'Tchaj-wan': 'TWN',
    'Togo': 'TGO',
    'Tokelau': 'TKL',
    'Tonga': 'TON',
    'Trinidad a Tobago': 'TTO',
    'Tunisko': 'TUN',
    'Turecko': 'TUR',
    'Turkmenistán': 'TKM',
    'Tuvalu': 'TUV',
    'Uganda': 'UGA',
    'Ukrajina': 'UKR',
    'Uruguay': 'URY',
    'Uzbekistán': 'UZB',
    'Vanuatu': 'VUT',
    'Vatikán': 'VAT',
    'Velká Británie': 'GBR',
    'Venezuela': 'VEN',
    'Vietnam': 'VNM',
    'Východní Timor': 'TLS',
    'Wallis a Futuna': 'WLF',
    'Zambie': 'ZMB',
    'Západní Sahara': 'ESH',
    'Zimbabwe': 'ZWE',
    'ČR': 'CZE',
    'Republika Kosovo': 'XKX',
    'Moldavská republika': 'MDA',
    'Spojené království': 'GBR',
    'Spojené státy': 'USA',
    'Jižní Afrika': 'ZAF',
    'bez státního občanství': 'XXA', # speciální kód pro osoby bez státního občanství
    'Nezjištěno': 'XXX', # custom kód pro nezjištěnou hodnotu
    'Kongo, demokrat.repub.': 'COD',
    'Mauricius': 'MUS',
    'Syrská arabská repub.': 'SYR',
    'Pobřeží Slonoviny': 'CIV',
    'Korejská republika': 'KOR',
    'Okupované palestinské území': 'PSE',
    'Korea, lidově dem. rep.': 'PRK',
    'Sv.Tomáš a Princův ostr': 'STP',
    'Brunej Darussalam': 'BRN',
    'Papua Nová Guinea': 'PNG',
    'Sv.Vincenc a Grenadiny': 'VCT'
}

# Aplikace mapování na sloupec country
df['country_iso'] = df['country'].map(country_mapping)

# Kontrola, zda některé země nebyly namapovány
missing_mapping = df[df['country_iso'].isna()]
if len(missing_mapping) > 0:
    print(f"Počet zemí bez mapování: {len(missing_mapping)}")
    print("Seznam zemí bez mapování:")
    print(missing_mapping["country"].unique())
else:
    print("Všechny země byly úspěšně namapovány na ISO kódy!")

# Nahrazení původního sloupce country ISO kódy
df['original_country'] = df['country']  # Uložení původních názvů pro pozdější referenci
df['country'] = df['country_iso']
df.drop('country_iso', axis=1, inplace=True)

# Zobrazení výsledku
print(df.head(10))

Počet unikátních zemí v datasetu: 189
Všechny země byly úspěšně namapovány na ISO kódy!
    gender       age country  year     original_country
158    muž    <0; 5)     DZA  2023             Alžírsko
160    muž  <35; 40)     AFG  2023          Afghánistán
161    muž  <30; 35)     AUT  2023             Rakousko
162    muž    <0; 5)     BEL  2023               Belgie
163    muž  <60; 65)     BEL  2023               Belgie
164    muž  <15; 20)     BIH  2023  Bosna a Hercegovina
165    muž  <65; 70)     DZA  2023             Alžírsko
166    muž  <35; 40)     AZE  2023          Ázerbájdžán
167    muž  <10; 15)     AUT  2023             Rakousko
168    muž  <35; 40)     BEL  2023               Belgie


In [14]:
# Vytvořit z df soubor csv
df = df[["country", "gender", "age", "year"]]
# Uložení do CSV
df.to_csv("cizinci_data.csv", index=False)